In [1]:
# run this first
# This will help you quickly gather base demand information from your system

from epyt import epanet
import pandas as pd
import numpy as np

# read in the relevant .inp file (assuming you have already created your pressure zones)
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/GillDataTransfer/GitHub/Box-Complex-Small-Systems/Whitesburg_GillUpdates.inp')
patternIndex = d.getNodeDemandPatternIndex()
baseDemand = d.getNodeBaseDemands()
patternIndex = patternIndex[1][:]
baseDemand = baseDemand[1][:]

df = pd.DataFrame([patternIndex, baseDemand])

#This has a place holder, ignore the first number reported below. Additionally, please remember that this is indexed.
# if the first pattern in your .inp file is named '2' you'll have to be mindful.
count = [0,0,0,0,0,0]

for i in range(6):
    for j in range(308):
        if df.iloc[0][j] == i:
            add = df.iloc[1][j]
            count[i] = count[i] + add

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File Whitesburg_GillUpdates.inp loaded successfully.



In [2]:
#The number of inputs here will have to change depending on the number of demand patterns you are calibrating

def massFlowRate(demandPattern1, demandPattern2):
    
    #For the sake of this example, this pattern is for Tunnel Hill
    demandPattern1 = demandPattern1
    #For the sake of this example, this pattern is for Sawmill
    demandPattern2 = demandPattern2
    import numpy as np

    #This is just so that we can have an initial flow rate coming from the treatment plant for our mass balance

    #Corresponds to Tunnel Hill
    d.setPattern(2, demandPattern1)
    #Corresponds to Sawmill Road
    d.setPattern(3, demandPattern2)

    d.openHydraulicAnalysis()
    d.initializeHydraulicAnalysis()
    Series = d.getComputedHydraulicTimeSeries()
    d.closeHydraulicAnalysis()

    #capturing the proper head
    Head = Series.Head[:,[306,307]]
    #capturing the proper flows (from the WTP)
    Flow = Series.Flow[:,341]

    #instead of getting the demands specifically for the unkown zones, in Whitesburg
    #We will take the flow rate into the zones from their respective pumps for accuracy
    #sake in the mass balance
    FlowBartesta = Series.Flow[:,158]
    FlowCowan = Series.Flow[:,334]
    FlowColley = Series.Flow[:,324]

    totalFlowFromUnknown = abs(FlowBartesta) + abs(FlowCowan) + abs(FlowColley)
    #capturing the time
    timeCheck = Series.Time[:]/3600


    resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], totalFlowFromUnknown], columns = timeCheck )

    #okay so flow needs to be processed a little more for Whitesburg. The reason for this is because if we turn a pump on at 1:15 the flow rate at 1:00
    # is 0. Thats an issue because for 45 minutes we have flow coming into the system that we are not accounting for. Therefore, we
    # need to find the average flow over the hour to plug into the mass balance equation. please follow logic below:

    #Here we are computing the average flow for each hour
    averageFlowRate = [0] * 24
    for h in range(23):
        hourlyTotalVolume = 0
        hourlyAverage = 0
        for idx, t in enumerate(timeCheck):
            if h <= t < (h + 1):
                timeDifference = (timeCheck[idx+1] - timeCheck[idx]) *60
                flowAtTime = Flow[idx]
                flowVolume = timeDifference * flowAtTime
                hourlyTotalVolume = hourlyTotalVolume + flowVolume
                hourlyAverage = hourlyTotalVolume/60
                #print(f"index {idx} belongs to hour {h}, with a difference of {timeDifference} minutes a running total flow volume of {flowVolume} and a running average of {hourlyAverage}")
        averageFlowRate[h] = hourlyAverage
        hourlyTotalVolume = 0


    averageFlowRate2 = [0] * 24
    for h in range(23):
        hourlyTotalVolume2 = 0
        hourlyAverage2 = 0
        for idx, t in enumerate(timeCheck):
            if h <= t < (h + 1):
                timeDifference = (timeCheck[idx+1] - timeCheck[idx]) *60
                flowAtTime = totalFlowFromUnknown[idx]
                flowVolume = timeDifference * flowAtTime
                hourlyTotalVolume2 = hourlyTotalVolume2 + flowVolume
                hourlyAverage2 = hourlyTotalVolume2/60
                #print(f"index {idx} belongs to hour {h}, with a difference of {timeDifference} minutes a running total flow volume of {flowVolume} and a running average of {hourlyAverage}")
        averageFlowRate2[h] = hourlyAverage2
        hourlyTotalVolume2 = 0
    
    return averageFlowRate, averageFlowRate2, resultDataFrame

In [ ]:
# Testing generic Box - Complex on Whitesburg Kentucky Data

from epyt import epanet
import numpy as np
import pandas as pd
import math

# Find the file path of the EPANET file in your directory
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/GillDataTransfer/GitHub/Box-Complex-Small-Systems/Whitesburg_GillUpdates.inp')

# Read in generic tank data, change the path to match your system
dfTanks = pd.read_excel('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/GillDataTransfer/GitHub/Box-Complex-Small-Systems/exampleTankLevels.xlsx')

# This is where we are storing the solutions for the demand factors
#For the sake of this example, this pattern is for Tunnel Hill
resultPattern1 = [1] * 24
#For the sake of this example, this pattern is for Sawmill
resultPattern2 = [1] * 24

averageFlowRate, averageFlowRate2, resultDataFrame = massFlowRate(resultPattern1, resultPattern2)

# the total length of the EPS
for h in range(8):
    
    # create random positive demand factors (here we will create n-1 factors where n is the number of zones in the system)
    demandFactors = [[1], [2], [4]]
    
    # change depending on how many tanks are in sim (this is for the tank levels at the end of the hour we are analyzing)
    sawmillRoad = dfTanks.iloc[h+1,0]
    tunnelHill = dfTanks.iloc[h+1,1]
    
    #Here we are putting a data frame together for our change in tanks levels for processing the mass balance
    
    flowSawmill = (((math.pi/4) * ( 50**2) * (resultDataFrame.iloc[1][h] - sawmillRoad)) * 7.48)/60
    flowTunnelHill = (((math.pi/4) * (25**2) * (resultDataFrame.iloc[2][h] - tunnelHill)) * 7.48)/60

    totalTankFlow = flowSawmill + flowTunnelHill
    
    # Whitesburg is weird, if I did (n-1) points then I would only have 2 of them so I am just creating a third point 
    # so that the box - complex works. That won't create any issues - we are still working in 2-D space (pattern 1 and 2) and 
    # creating a third pattern based off the mass balance
    
    pointsInSimplex = [1, 2, 3]
    
    averageFlowRate, averageFlowRate2, resultDataFrame = massFlowRate(resultPattern1, resultPattern2)
    
    #change depending on the number of points in the simplex
    for i in range(3):

            # depending on the number of zones, create demand patterns
            resultPattern1[h] = demandFactors[i][0]
            resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate2[h] + (count[2] * resultPattern1[h]))) / count[3]
            
            
            # this is complicated but because we don't have actual flow data, changing patterns changes the flow rate out of the
            # plant and therefore we need to account for that variability
            
        # Here we first discover why we are using (n-1) points in the simplex. Because we want to incorporate real data, we are using
        # a mass balance to determine what the nth demand pattern will be. Add or subrtract patterns to ensure that this suits the system.

            # nthPattern[h] = ((volume in at time t + the change in storage of the tanks(flow out is positive)) - (Base Demand Zone 1 * pattern1) - (Base Demand Zone 2 * pattern2) - (Base Demand zone 3 * pattern3)) / Base Demand nth zone

            # Create appropriate number of demand patterns dependant on system. Make sure this indexes properly within the .inp file.
            # for example d.setPattern(1, [pattern1] + dummyPattern) is calling the first indexed demand pattern in the file and not 
            # necessarily the demand pattern with the name of 1.

            d.setPattern(2, resultPattern1)
            d.setPattern(3, resultPattern2)
            
            
            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()
            
            #capturing the proper head
            Head = Series.Head[:,[306,307]]
            #capturing the proper flows (from the WTP)
            Flow = Series.Flow[:,341]

            #instead of getting the demands specifically for the unkown zones, in Whitesburg
            #We will take the flow rate into the zones from their respective pumps for accuracy
            #sake in the mass balance
            FlowBartesta = Series.Flow[:,158]
            FlowCowan = Series.Flow[:,334]
            FlowColley = Series.Flow[:,324]

            totalFlowFromUnknown = abs(FlowBartesta) + abs(FlowCowan) + abs(FlowColley)
            #capturing the time
            timeCheck = Series.Time[:]/3600


            resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], totalFlowFromUnknown], columns = timeCheck )

            # Change the below statements (add or subtract) dpending on how many tanks are in your objective function. 
            # You must also go into the .inp file to figure out the exact index of the tanks in the sim (you can use d.getNodeNameID)
            ModelTank1 = resultDataFrame.iloc[1][h+1]
            ModelTank2 = resultDataFrame.iloc[2][h+1]
            
           

            # Update the objective function to match the number of tanks that are in the simulation. 
            Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2) 
            pointsInSimplex[i] = Error
            simplexCounter = 0
    while ((abs(pointsInSimplex[0] >=.01)) or (abs(pointsInSimplex[1] >=.01)) or (abs(pointsInSimplex[2] >=.01))) and simplexCounter <= 25:
        
        simplexCounter = simplexCounter + 1
        averageFlowRate, averageFlowRate2, resultDataFrame = massFlowRate(resultPattern1, resultPattern2)
         
        # Change range depending on the number of points in the simplex
        maxVal, whereMax = max(pointsInSimplex), pointsInSimplex.index(max(pointsInSimplex))
        logical = [i for i in range(3) if pointsInSimplex[i] != maxVal]

        ph = demandFactors[whereMax]

        #add another "demandFactors[logical[0,1,2...]][j]" if number of zones change. Also change the denominator as well as the "in range() statement"
        centroid = [(demandFactors[logical[0]][j] + demandFactors[logical[1]][j]) / 2 for j in range(1)]


        # change the range to match the number of zones minus 1 (n-1)
        newPoint = [(2.5 * centroid[j]) - (1.5 * ph[j]) for j in range(1)]

        #check to see if evaluated nth pattern is greater than 0. Change the "newPoint[]" and add more if the number of zones is increased
        resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate[h] + (count[2] * newPoint[0]))) / count[3]

        # Change the size of "newPoint" to match the number of zones minus 1 (n-1)
        while newPoint[0] < 0 or resultPattern2[h] < 0:

            # change the range to match the number of zones minus 1 (n-1)
            newPoint = [(0.5 * newPoint[j]) + (0.5 * centroid[j]) for j in range(1)]

            #check to see if evaluated nth pattern is greater than 0. Change the "newPoint[]" and add more if the number of zones is increased
            resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate2[h] + (count[2] * newPoint[0]))) / count[3]

        # change the number of patterns to match the number of zones. If mass balance remove the comment
        resultPattern1[h] = newPoint[0]
        resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate2[h] + (count[2] * newPoint[0]))) / count[3]

        d.setPattern(2, resultPattern1)
        d.setPattern(3, resultPattern2)


        d.openHydraulicAnalysis()
        d.initializeHydraulicAnalysis()
        Series = d.getComputedHydraulicTimeSeries()
        d.closeHydraulicAnalysis()

        #capturing the proper head
        Head = Series.Head[:,[306,307]]
        #capturing the proper flows (from the WTP)
        Flow = Series.Flow[:,341]

        #instead of getting the demands specifically for the unkown zones, in Whitesburg
        #We will take the flow rate into the zones from their respective pumps for accuracy
        #sake in the mass balance
        FlowBartesta = Series.Flow[:,158]
        FlowCowan = Series.Flow[:,334]
        FlowColley = Series.Flow[:,324]

        totalFlowFromUnknown = abs(FlowBartesta) + abs(FlowCowan) + abs(FlowColley)
        #capturing the time
        timeCheck = Series.Time[:]/3600


        resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], totalFlowFromUnknown], columns = timeCheck )

        #change this to match the tank heads if you change systems
        ModelTank1 = resultDataFrame.iloc[1][h+1]
        ModelTank2 = resultDataFrame.iloc[2][h+1]

        Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2) 

        if Error < maxVal:
            #change the "resultPattern1..resultPatternN" to match the number of zones minus 1 (n-1)
            demandFactors[whereMax] = [resultPattern1[h]]
            pointsInSimplex[whereMax] = Error
        else:
            
            # change the range to match the number of zones minus 1 (n-1)
            newPoint = [(0.5 * ph[j]) + (0.5 * centroid[j]) for j in range(1)]

            # Evaluate the new point

            resultPattern1[h] = newPoint[0]
            resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate2[h] + (count[2] * newPoint[0]))) / count[3]

            d.setPattern(2, resultPattern1)
            d.setPattern(3, resultPattern2)


            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()

            #capturing the proper head
            Head = Series.Head[:,[306,307]]
            #capturing the proper flows (from the WTP)
            Flow = Series.Flow[:,341]

            #instead of getting the demands specifically for the unkown zones, in Whitesburg
            #We will take the flow rate into the zones from their respective pumps for accuracy
            #sake in the mass balance
            FlowBartesta = Series.Flow[:,158]
            FlowCowan = Series.Flow[:,334]
            FlowColley = Series.Flow[:,324]

            totalFlowFromUnknown = abs(FlowBartesta) + abs(FlowCowan) + abs(FlowColley)
            #capturing the time
            timeCheck = Series.Time[:]/3600


            resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], totalFlowFromUnknown], columns = timeCheck )

            ModelTank1 = resultDataFrame.iloc[1][h+1]
            ModelTank2 = resultDataFrame.iloc[2][h+1]

            Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2)


            #change the "resultPattern1..resultPatternN" to match the number of zones minus 1 (n-1)
            demandFactors[whereMax] = [resultPattern1[h]]
            pointsInSimplex[whereMax] = Error
            
    minVal, whereMin = min(pointsInSimplex), pointsInSimplex.index(min(pointsInSimplex))
    #store the calibrated demand factors from this hour
    resultPattern1[h] = demandFactors[whereMin][0]
    resultPattern2[h] = (averageFlowRate[h] + (totalTankFlow) - (averageFlowRate2[h] + (count[2] * resultPattern1[h]))) / count[3]
        
        
    print('Demand for hour',(h+1), 'is', resultPattern1[h], 'for tunnelHill and', resultPattern2[h], 'for Sawmill Road and with a total MSE of', min(pointsInSimplex))
    

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File Whitesburg_GillUpdates.inp loaded successfully.

[0.0003489612172056114, 1.1768954947882706, 0.36561083867670585]
[0.0003489612172056114, 0.1859696679544448, 0.36561083867670585]
[0.0003489612172056114, 0.1859696679544448, 0.05533842537362218]
[0.0003489612172056114, 0.020934475976263932, 0.05533842537362218]
[0.0003489612172056114, 0.020934475976263932, 0.007448051896705304]
Demand for hour 1 is 6.0 for tunnelHill and 0.29436445729156735 for Sawmill Road and with a total MSE of 0.0003489612172056114
[0.016375877162527076, 0.002253562908438525, 0.00014249715972083866]
Demand for hour 2 is 7.9897308349609375 for tunnelHill and 0.22487541292248647 for Sawmill Road and with a total MSE of 0.00014249715972083866
Demand for hour 3 is 8.505775719881058 for tunnelHill and 0.34941741872331816 for Sawmill Road and with a total MSE of 6.820128460672281e-05
[0.004262425595057756, 0.004649040796725804, 0.06549208125580348]
Demand for hou

In [1]:
pip install epyt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
Hello = 'Hello'